In [1]:
import os
import pandas as pd
from lxml import etree
import datetime
from datetime import datetime
import time


ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}



def readfiles(pathtofiles):
    eventslist = []
    for filename in os.listdir(pathtofiles):
        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        creationdate = regestroot.find(".//tei:creation/tei:date", ns)
        origindate = regestroot.find(".//tei:history/tei:origin", ns)
        if creationdate is not None:
           date = createdate(creationdate)
        elif origindate is not None:
           date = createdate(origindate)
        else:
           date = " "
        eventNodes = regestroot.xpath(".//*[@type='event'][./tei:catchwords/@n='disp']", namespaces=ns)
        if len(eventNodes) > 0:
            print("--------------------------")
            print('filename: ',filename)
            print('datum: ',date)
            for eventNode in eventNodes:
                print('event: ' ,eventNode.attrib['ref'])
                eventsDict = {}
                eventsDict['xml'] = filename
                eventsDict['date'] = date
                catchwordslist =[]
                eventsDict['event_ref'] = eventNode.attrib['ref']
                catchwords = eventNode.xpath(".//tei:catchwords[@n='disp']", namespaces=ns)
                for catchword in catchwords:
                 #   print('catchword: ',catchword.xpath("normalize-space(.//text())"))
                    catchwordslist.append(catchword.xpath("normalize-space(.//text())"))
                catchwordstring = " | ".join(catchwordslist)
                print("catchwords: ", catchwordstring)
                eventsDict['catchwords'] = catchwordstring
                eventslist.append(eventsDict)
    createcoloumns(eventslist)

def createcoloumns(eventslist):
    listoferrors ={}

    catchwordsFrame = pd.DataFrame()
    xmls = []
    dates = []
    events =[]
    catchwords =[]

    for event in eventslist:
        eventerror = []
        xmls.append(event['xml'])
        try:
            dates.append(event['date'][0])
        except:
            dates.append(' ')
            eventerror.append('Datum Fehlerhaft')

        try:
            events.append(event['event_ref'])
        except:
            print(event)
            events.append(' ')
            eventerror.append('Eventref Fehlerhaft')

        try:
            catchwords.append(event['catchwords'])
        except:
            catchwords.append(' ')
            eventerror.append('Catchwords fehlerhaft')

        if len(eventerror) > 0:
            listoferrors[event['xml']] = eventerror

    writeerrorfile(listoferrors)

    catchwordsFrame['xml'] = xmls
    catchwordsFrame['date'] = dates
    catchwordsFrame['event'] = events
    catchwordsFrame['catchwords'] = catchwords

    catchwordsFrame.to_csv("../data/output/csv/extractedCatchwords.csv", sep="\t", encoding="utf-8")

    writer = pd.ExcelWriter('../data/output/xlsx/extractedCatchwords.xlsx')
    catchwordsFrame.to_excel(writer, 'ExtractedCatchwords', index=False)
    writer.save()


def writeerrorfile(listoferrors):
    with open("../data/output/catchwords_ERROR.txt", "w") as file:

        if len(listoferrors) >0:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\n")
            for xml in listoferrors.keys():
                file.write("--------------\n")
                file.write("file: " + xml+"\n")
                for error in listoferrors[xml]:
                    file.write(error+"\n")

        else:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\nKeine ERRORS gefunden")


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return [date, year]
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]

readfiles("../../sources/QGW/regests_QGW_1370-1404/done")

--------------------------
filename:  1000.xml
datum:  [datetime.date(1381, 3, 26), '1381']
event:  ev__QGW_II_I_1000
catchwords:  verkaufen
--------------------------
filename:  1001.xml
datum:  [datetime.date(1381, 4, 29), '1381']
event:  ev__QGW_II_I_1001
catchwords:  beurkundet, dass vor ihm in offenem Gerichte | versessens diensts und die darauf ertheilten zwispild geklagt habe | gibt | als Beschauleute mit | macht ihn
--------------------------
filename:  1002.xml
datum:  [datetime.date(1381, 5, 6), '1381']
event:  ev__QGW_II_I_1002
catchwords:  verbietet | gerichtlich angehalten und wegen der Schulden und Vergehen Anderer abgeurtheilt werden
--------------------------
filename:  1003.xml
datum:  [datetime.date(1381, 5, 6), '1381']
event:  ev__QGW_II_I_1003
catchwords:  verbietet | durch ungewohnte und übertriebene Abgaben | zu belästigen
--------------------------
filename:  1004.xml
datum:  [datetime.date(1381, 5, 6), '1381']
event:  ev__QGW_II_I_1004
catchwords:  bestätigt | u

catchwords:  bestätigen | gekauft zu haben | verpflichten sich | ihn in ordentlichem Bau zu halten
--------------------------
filename:  1057.xml
datum:  [datetime.date(1383, 9, 7), '1383']
event:  ev__QGW_II_I_1057
catchwords:  bestätigen | gekauft zu haben | verpflichten sich | ihn in ordentlichem Bau zu halten
--------------------------
filename:  1058.xml
datum:  [datetime.date(1383, 9, 7), '1383']
event:  ev__QGW_II_I_1058
catchwords:  bestätigen | gekauft zu haben | verpflichten sich | ihn in ordentlichem Bau zu halten
--------------------------
filename:  1059.xml
datum:  [datetime.date(1383, 9, 7), '1383']
event:  ev__QGW_II_I_1059
catchwords:  bestätigen | gekauft zu haben | verpflichten sich | ihn in ordentlichem Bau zu halten
--------------------------
filename:  1060.xml
datum:  [datetime.date(1383, 11, 27), '1383']
event:  ev__QGW_II_I_1060
catchwords:  geben
--------------------------
filename:  1061.xml
datum:  [datetime.date(1384, 2, 4), '1384']
event:  ev__QGW_II_I_106

--------------------------
filename:  1108.xml
datum:  [datetime.date(1386, 10, 13), '1386']
event:  ev__QGW_II_I_1108
catchwords:  stellen | verpflichten sich zur Rückerstattung
--------------------------
filename:  1109.xml
datum:  [datetime.date(1386, 11, 8), '1386']
event:  ev__QGW_II_I_1109
catchwords:  verzichten zu Gunsten
--------------------------
filename:  1110.xml
datum:  [datetime.date(1386, 12, 20), '1386']
event:  ev__QGW_II_I_1110
catchwords:  überlassen | zu freier Verfügung
--------------------------
filename:  1111.xml
datum:  [datetime.date(1386, 12, 24), '1386']
event:  ev__QGW_II_I_1111
catchwords:  stellen | über die Leistung des Hofzinses für die | setzen
--------------------------
filename:  1112.xml
datum:  [datetime.date(1387, 1, 5), '1387']
event:  ev__QGW_II_I_1112
catchwords:  verkaufen
--------------------------
filename:  1113.xml
datum:  [datetime.date(1387, 2, 8), '1387']
event:  ev__QGW_II_I_1113
catchwords:  stellen | verpfänden
---------------------

filename:  1159.xml
datum:  [datetime.date(1388, 8, 21), '1388']
event:  ev__QGW_II_I_1159
catchwords:  verkauft
--------------------------
filename:  1161.xml
datum:  [datetime.date(1388, 8, 29), '1388']
event:  ev__QGW_II_I_1161
catchwords:  befiehlt, dass
--------------------------
filename:  1162.xml
datum:  [datetime.date(1388, 9, 14), '1388']
event:  ev__QGW_II_I_1162
catchwords:  beurkundet | schulde | verpfändet
--------------------------
filename:  1163.xml
datum:  [datetime.date(1388, 9, 15), '1388']
event:  ev__QGW_II_I_1163
catchwords:  verkaufen
--------------------------
filename:  1164.xml
datum:  [datetime.date(1388, 9, 21), '1388']
event:  ev__QGW_II_I_1164
catchwords:  schreibt
--------------------------
filename:  1165.xml
datum:  [datetime.date(1388, 10, 4), '1388']
event:  ev__QGW_II_I_1165
catchwords:  verleiht
--------------------------
filename:  1166.xml
datum:  [datetime.date(1388, 10, 6), '1388']
event:  ev__QGW_II_I_1166
catchwords:  verkauft
---------------

filename:  1279.xml
datum:  [datetime.date(1394, 3, 27), '1394']
event:  ev__QGW_II_I_1279
catchwords:  verkaufen
--------------------------
filename:  1280.xml
datum:  [datetime.date(1394, 3, 29), '1394']
event:  ev__QGW_II_I_1280
catchwords:  verkaufen wegen des Klosters grosere schuld und scheden
--------------------------
filename:  1281.xml
datum:  [datetime.date(1394, 5, 2), '1394']
event:  ev__QGW_II_I_1281
catchwords:  bestätigen | ausbezahlt habe | verpflichten sich | zurückzuerstatten
--------------------------
filename:  1282.xml
datum:  [datetime.date(1394, 5, 11), '1394']
event:  ev__QGW_II_I_1282
catchwords:  verkaufen
--------------------------
filename:  1283.xml
datum:  [datetime.date(1394, 6, 5), '1394']
event:  ev__QGW_II_I_1283
catchwords:  verkaufen
--------------------------
filename:  1284.xml
datum:  [datetime.date(1394, 6, 29), '1394']
event:  ev__QGW_II_I_1284
catchwords:  bestätigen | Bürgschaft | verpflichten sich, jene beiden ohne jeglichen Schaden von der 

...

--------------------------
filename:  1558.xml
datum:  [datetime.date(1403, 9, 29), '1403']
event:  ev__QGW_II_I_1558
catchwords:  schafft | zu einem Jahrtage
--------------------------
filename:  1559.xml
datum:  [datetime.date(1403, 10, 4), '1403']
event:  ev__QGW_II_I_1559
catchwords:  bestätigt | alles Gut | erhalten zu haben
--------------------------
filename:  1560.xml
datum:  [datetime.date(1403, 10, 5), '1403']
event:  ev__QGW_II_I_1560
catchwords:  beurkundet seinen letzten Willen
--------------------------
filename:  1561.xml
datum:  [datetime.date(1403, 10, 10), '1403']
event:  ev__QGW_II_I_1561
catchwords:  bestätigt | erhalten zu haben
--------------------------
filename:  1562.xml
datum:  [datetime.date(1403, 10, 12), '1403']
event:  ev__QGW_II_I_1562
catchwords:  verkaufen
--------------------------
filename:  1563.xml
datum:  [datetime.date(1403, 11, 10), '1403']
event:  ev__QGW_II_I_1563
catchwords:  beurkundet ihren letzten Willen
--------------------------
filename:

datum:  [datetime.date(1371, 3, 9), '1371']
event:  ev__QGW_II_I_787
catchwords:  verkaufen
--------------------------
filename:  788.xml
datum:  [datetime.date(1371, 3, 13), '1371']
event:  ev__QGW_II_I_788
catchwords:  verkaufen
--------------------------
filename:  789.xml
datum:  [datetime.date(1371, 4, 13), '1371']
event:  ev__QGW_II_I_789
catchwords:  verkauft aus ehafter, vor dem Rathe bewiesener Noth
--------------------------
filename:  790.xml
datum:  [datetime.date(1371, 5, 3), '1371']
event:  ev__QGW_II_I_790
catchwords:  bestätigen | stiften | widmen
--------------------------
filename:  791.xml
datum:  [datetime.date(1371, 5, 15), '1371']
event:  ev__QGW_II_I_791
catchwords:  beurkundet, dass vor ihm in offenem Gerichte | geklagt habe | macht
--------------------------
filename:  792.xml
datum:  [datetime.date(1371, 5, 26), '1371']
event:  ev__QGW_II_I_792
catchwords:  geben
--------------------------
filename:  793.xml
datum:  [datetime.date(1371, 7, 8), '1371']
event:  

--------------------------
filename:  861.xml
datum:  [datetime.date(1375, 7, 12), '1375']
event:  ev__QGW_II_I_861
catchwords:  bestätigt | stellt einen Revers
--------------------------
filename:  862.xml
datum:  [datetime.date(1375, 7, 21), '1375']
event:  ev__QGW_II_I_862
catchwords:  verkaufen
--------------------------
filename:  863.xml
datum:  [datetime.date(1375, 8, 2), '1375']
event:  ev__QGW_II_I_863
catchwords:  verkauft
--------------------------
filename:  864.xml
datum:  [datetime.date(1375, 9, 27), '1375']
event:  ev__QGW_II_I_864
catchwords:  verkauft
--------------------------
filename:  865.xml
datum:  [datetime.date(1375, 11, 7), '1375']
event:  ev__QGW_II_I_865
catchwords:  verkaufen
--------------------------
filename:  866.xml
datum:  [datetime.date(1375, 11, 25), '1375']
event:  ev__QGW_II_I_866
catchwords:  gibt
--------------------------
filename:  867.xml
datum:  [datetime.date(1375, 11, 25), '1375']
event:  ev__QGW_II_I_867
catchwords:  verkaufen
-----------

filename:  929.xml
datum:  [datetime.date(1378, 2, 23), '1378']
event:  ev__QGW_II_I_929
catchwords:  bestätigt die Stiftung | bestimmt, dass die Präsentation | zustehen solle
--------------------------
filename:  930.xml
datum:  [datetime.date(1378, 3, 2), '1378']
event:  ev__QGW_II_I_930
catchwords:  beurkunden | verkauft haben | verpflichten
--------------------------
filename:  931.xml
datum:  [datetime.date(1378, 3, 2), '1378']
event:  ev__QGW_II_I_931
catchwords:  bestätigt | zurückbezahlt habe | begleichen solle
--------------------------
filename:  932.xml
datum:  [datetime.date(1378, 3, 8), '1378']
event:  ev__QGW_II_I_932
catchwords:  gibt | zu rechter Morgengabe
--------------------------
filename:  933.xml
datum:  [datetime.date(1378, 3, 12), '1378']
event:  ev__QGW_II_I_932
catchwords:  gibt
--------------------------
filename:  934.xml
datum:  [datetime.date(1378, 4, 4), '1378']
event:  ev__QGW_II_I_934
catchwords:  verbietet | zu belästigen
--------------------------
fil

--------------------------
filename:  997.xml
datum:  [datetime.date(1381, 1, 18), '1381']
event:  ev__QGW_II_I_997
catchwords:  treffen in dem Streite | Anspruch erhebt | folgende Entscheidung | zugesprochen | nach Recht erben soll
--------------------------
filename:  998.xml
datum:  [datetime.date(1381, 2, 14), '1381']
event:  ev__QGW_II_I_998
catchwords:  bestätigen den Empfang | als Antheil
--------------------------
filename:  999.xml
datum:  [datetime.date(1381, 3, 11), '1381']
event:  ev__QGW_II_I_999
catchwords:  überlässt | bis zum nächsten Mathiastage die Nutzniessung zweier Höfe und der dazu gehörigen Gründe | zu entrichten hat
